# Use Model Context Protocol (MCP) as tools with Strands Agent

## Overview
The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands AI SDK integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. The Strands Agent SDK includes built-in support for connecting to MCP servers and using their tools.

In this example we will show you how to use MCP tools on your Strands Agent. We will use the [AWS Documentation MCP server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/) which provides tools to access AWS documentation, search for content, and get recommendations. This MCP server has 3 main features:

- **Read Documentation**: Fetch and convert AWS documentation pages to markdown format
- **Search Documentation**: Search AWS documentation using the official search API
- **Recommendations**: Get content recommendations for AWS documentation pages



## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Feature used        |MCP Tools                                          |
|Agent Structure     |Single agent architecture                          |

</div>

## Architecture

<div style="text-align:center">
    <img src="images/architecture.png" width="65%" />
</div>

## Key Features
* **Single agent architecture**: this example creates a single agent that interacts with MCP tools
* **MCP tools**: Integration of MCP tools with your agent

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 3.7 enabled on Amazon Bedrock

Let's now install the requirement packages for our Strands Agent agent

In [1]:
# installing pre-requisites
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 193.3 MB/s  0:00:00


### Importing dependency packages

Now let's import the dependency packages

In [2]:
import threading
import time
from datetime import timedelta

from mcp import StdioServerParameters, stdio_client
from mcp.client.streamable_http import streamablehttp_client
from mcp.server import FastMCP
from strands import Agent
from strands.tools.mcp import MCPClient

### Connect to MCP server using stdio transport

[Transports](https://modelcontextprotocol.io/specification/2025-03-26/basic/transports) in MCP provide the foundations for communication between clients and servers. It handles the underlying mechanics of how messages are sent and received. At the moment there are three standards transport implementations built-in in MCP:

- **Standard Input/Output (stdio)**: enables communication through standard input and output streams. It is particularly useful for local integrations and command-line tools
- **Streamable HTTP**: this replaces the HTTP+SSE transport from previous protocol version. In the Streamable HTTP transport, the server operates as an independent process that can handle multiple client connections. This transport uses HTTP POST and GET requests. Server can optionally make use of Server-Sent Events (SSE) to stream multiple server messages. This permits basic MCP servers, as well as more feature-rich servers supporting streaming and server-to-client notifications and requests.
- **SSE**: legacy transport for HTTP-based MCP servers that use Server-Sent Events transport  

Overall, you should use stdio for building command-line tools, implementing local integrations and working with shell scripts. You should use Streamable HTTP transports when you need a flexible and efficient way for AI agents to communicate with tools and services, especially when dealing with stateless communication or when minimizing resource usage is crucial.

You can also use **custom transports** implementation for your specific needs. 


Let's now connect to the MCP server using stdio transport. First of all, we will use the class `MCPClient` to connect to the [AWS Documentation MCP Server](https://awslabs.github.io/mcp/servers/aws-documentation-mcp-server/). This server provides tools to access AWS documentation, search for content, and get recommendations.

In [3]:
# Connect to an MCP server using stdio transport
stdio_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `stdio_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [4]:
# Create an agent with MCP tools
with stdio_mcp_client:
    # Get the tools from the MCP server
    tools = stdio_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent("What is Amazon Bedrock pricing model. Be concise.")

I'll look up the Amazon Bedrock pricing information for you. Let me search for the official documentation.
Tool #1: search_documentation
Now let me fetch the detailed pricing information from the Amazon Bedrock documentation:
Tool #2: read_documentation
Let me get more information on the Provisioned Throughput aspect:
Tool #3: read_documentation
Let me also look at information about calculating the cost of custom models:
Tool #4: read_documentation
Based on the AWS documentation, here's a concise summary of Amazon Bedrock's pricing model:

# Amazon Bedrock Pricing Model

1. **Pay-as-you-go Model**:
   - Pay based on the volume of input and output tokens processed
   - Different rates apply for different foundation models
   - No upfront commitments required for basic usage

2. **Provisioned Throughput Option**:
   - Purchase dedicated capacity for higher throughput at a fixed cost
   - Billed hourly based on:
     - The model selected
     - Number of Model Units (MUs) purchased
     -

### Connect to MCP server using Streamable HTTP

Let's now connect to the MCP server using Streamable HTTP transport. First let's start a simple MCP server using Streamable HTTP transport. 

For this example we will create our own MCP server. The architecture will look as following

<div style="text-align:center">
    <img src="images/architecture_2.png" width="65%" />
</div>

In [5]:
# Create an MCP server
mcp = FastMCP("Calculator Server")

# Define a tool


@mcp.tool(description="Calculator tool which performs calculations")
def calculator(x: int, y: int) -> int:
    return x + y


@mcp.tool(description="This is a long running tool")
def long_running_tool(name: str) -> str:
    time.sleep(25)
    return f"Hello {name}"


def main():
    mcp.run(transport="streamable-http", mount_path="mcp")

Let's now start a thread with the `streamable-http` server

In [6]:
thread = threading.Thread(target=main)
thread.start()

#### Integrating Streamable HTTP client with Agent

Now let's use `streamablehttp_client` integrate this server with a simple agent. 

In [7]:
def create_streamable_http_transport():
    return streamablehttp_client("http://localhost:8000/mcp")


streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

#### Setup agent configuration and invoke it

Next we will set our agent configuration using the tools from the `streamable_http_mcp_client` object we just created. To do so, we need to list the tools available in the MCP server. We can use the `list_tools_sync` method for it. 

After that, we will ask a question to our agent.

In [8]:
with streamable_http_mcp_client:
    tools = streamable_http_mcp_client.list_tools_sync()

    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = str(agent("What is 2 + 2?"))

INFO:     Started server process [4196]
INFO:     Waiting for application startup.


[12/17/25 06:44:33] INFO     StreamableHTTP session manager started                  ]8;id=46448;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=217698;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#110\110]8;;\

INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


                    INFO     Created new transport with session ID:                  ]8;id=957457;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=867861;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             f36fc3e6401242a890bb457eef456a41                                                      

INFO:     127.0.0.1:36204 - "POST /mcp HTTP/1.1" 200 OK


...

                    INFO     Processing request of type CallToolRequest                               ]8;id=350102;file:///opt/conda/lib/python3.12/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=234294;file:///opt/conda/lib/python3.12/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\

The answer to 2 + 2 is 4.

[12/17/25 06:44:35] INFO     Terminating session: f36fc3e6401242a890bb457eef456a41           ]8;id=468746;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=44344;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py#646\646]8;;\

INFO:     127.0.0.1:36244 - "DELETE /mcp HTTP/1.1" 200 OK


                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=538676;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=138706;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

### Direct Tool Invocation

While tools are typically invoked by the agent based on user requests, you can also call MCP tools directly. This can be useful for workflow scenarios where you orchestrate multiple tools together.

In [9]:
query = {"x": 10, "y": 20}

with streamable_http_mcp_client:
    # direct tool invocation
    result = streamable_http_mcp_client.call_tool_sync(
        tool_use_id="tool-123", name="calculator", arguments=query
    )

    # Process the result
    print(f"Calculation result: {result['content'][0]['text']}")

                    INFO     Created new transport with session ID:                  ]8;id=615277;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=514735;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             aa67ebb88f96481dab0603f21223f649                                                      

INFO:     127.0.0.1:36256 - "POST /mcp HTTP/1.1" 200 OK


                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=698107;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=924603;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: aa67ebb88f96481dab0603f21223f649           ]8;id=267795;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=30144;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=457466;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=365155;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#146\146]8;;\

...

                    INFO     Processing request of type ListToolsRequest                              ]8;id=654863;file:///opt/conda/lib/python3.12/site-packages/mcp/server/lowlevel/server.py\server.py]8;;\:]8;id=436276;file:///opt/conda/lib/python3.12/site-packages/mcp/server/lowlevel/server.py#674\674]8;;\

Calculation result: 30


                    INFO     Terminating session: aa67ebb88f96481dab0603f21223f649           ]8;id=764590;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=26812;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py#646\646]8;;\

INFO:     127.0.0.1:36286 - "DELETE /mcp HTTP/1.1" 200 OK


                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=230672;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=291741;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

You can optionally also provide `read_timeout_seconds` while calling an MCP server tool to avoid it running for too long

In [10]:
with streamable_http_mcp_client:
    try:
        result = streamable_http_mcp_client.call_tool_sync(
            tool_use_id="tool-123",
            name="long_running_tool",
            arguments={"name": "Amazon"},
            read_timeout_seconds=timedelta(seconds=30),
        )

        if result["status"] == "error":
            print(f"Tool execution failed: {result['content'][0]['text']}")
        else:
            print(f"Tool execution succeeded: {result['content'][0]['text']}")
    except Exception as e:
        print(f"Tool call timed out or failed: {str(e)}")

                    INFO     Created new transport with session ID:                  ]8;id=170608;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py\streamable_http_manager.py]8;;\:]8;id=930346;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http_manager.py#233\233]8;;\
                             86615851adaf49f3a2f81cf21ddae47b                                                      

INFO:     127.0.0.1:36300 - "POST /mcp HTTP/1.1" 200 OK


                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=704127;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=839346;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 86615851adaf49f3a2f81cf21ddae47b           ]8;id=313454;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=687272;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=414232;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=524307;file:///opt/conda/lib/python3.12/site-packages/mcp/client/streamable_http.py#146\146]8;;\

...

                    INFO     HTTP Request: POST http://localhost:8000/mcp "HTTP/1.1 200 OK"         ]8;id=411699;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=701718;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

Tool execution succeeded: Hello Amazon


                    INFO     Terminating session: 86615851adaf49f3a2f81cf21ddae47b           ]8;id=157263;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py\streamable_http.py]8;;\:]8;id=566146;file:///opt/conda/lib/python3.12/site-packages/mcp/server/streamable_http.py#646\646]8;;\

INFO:     127.0.0.1:55156 - "DELETE /mcp HTTP/1.1" 200 OK


                    INFO     HTTP Request: DELETE http://localhost:8000/mcp "HTTP/1.1 200 OK"       ]8;id=278934;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=641423;file:///opt/conda/lib/python3.12/site-packages/httpx/_client.py#1740\1740]8;;\

### Interacting with multiple MCP servers

With Strands Agents you can also interact with multiple MCP servers using the same agent and configure tools setups such as the max number of tools that can be used in parallel (`max_parallel_tools`). Let's create a new agent to showcase this configuration:

<div style="text-align:center">
    <img src="images/architecture_3.png" width="85%" />
</div>

In this agent, we will again use the AWS Documentation MCP server and we will also use the [AWS CDK MCP Server](https://awslabs.github.io/mcp/servers/cdk-mcp-server/) which helps with AWS Cloud Development Kit (CDK) best practices, infrastructure as code patterns and security compliance with CDK Nag.

First let's connect to the two MCP servers using the stdio transport

In [11]:
# Connect to an MCP server using stdio transport
aws_docs_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(
            command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]
        )
    )
)

# Connect to an MCP server using stdio transport
cdk_mcp_client = MCPClient(
    lambda: stdio_client(
        StdioServerParameters(command="uvx", args=["awslabs.cdk-mcp-server@latest"])
    )
)

#### Create Agent with MCP servers

Next we will create the agent with the tools from both MCP servers

In [12]:
# Create an agent with MCP tools
with aws_docs_mcp_client, cdk_mcp_client:
    # Get the tools from the MCP server
    tools = aws_docs_mcp_client.list_tools_sync() + cdk_mcp_client.list_tools_sync()

    # Create an agent with these tools
    agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        tools=tools)

    response = agent(
        "What is Amazon Bedrock pricing model. Be concise. Also what are the best practices related to CDK?"
    )

I'll answer both parts of your question - Amazon Bedrock pricing and CDK best practices.

## Amazon Bedrock Pricing

Let me search for information on Amazon Bedrock's pricing model:
Tool #1: search_documentation
Now, let me retrieve the Amazon Bedrock pricing information:
Tool #2: read_documentation
Let me also check the information about provisioned throughput:
Tool #3: read_documentation
Now, let me get information on CDK best practices:
Tool #4: CDKGeneralGuidance
## Amazon Bedrock Pricing Model (Concise)

Amazon Bedrock uses a pay-as-you-go pricing model based on:

1. **Token-based pricing**: You pay for the volume of input and output tokens processed during model inference.

2. **Provisioned Throughput option**: 
   - Alternative to on-demand pricing for guaranteed capacity
   - Billed hourly at a fixed cost
   - Price depends on model, number of Model Units (MUs), and commitment duration
   - Commitment options: No commitment, 1 month, or 6 months (longer commitments offer discou

### Congratulations!

In this notebook you learned how to connect with MCP servers using Strands Agent and two MCP transport protocols: stdio and Streamable HTTP. You also learned how to connect multiple MCP servers to the same agent. Next, let's see how to use different models with your agent